In [1]:
import os
os.chdir("../..")
import json
import matplotlib.pyplot as plt
import pandas as pd
import requests
# !pip install google-api-python-client
from googleapiclient.discovery import build

# local import
from scripts.python.config import GoogleAPIkey

In [2]:
def service(developerKey: str):
    return build('trends', 'v1beta', developerKey=developerKey)


def get(method, params, developerKey):
    conn = service(developerKey)
    if method not in ['getGraph', 'getTimelinesForHealth']:
        raise NotImplementedError("Method not supported.")
    else:
        return getattr(conn, method)(**params).execute()

## Case-sensitive testing

In [3]:
upper_parms = {
    'terms': "Papua New Guinea",
    'restrictions_geo': "AU",
    'restrictions_startDate': "2004-01"
}

# change terms to lower case
lower_parms = upper_parms.copy()
lower_parms["terms"] = lower_parms["terms"].lower()

upper = get(method="getGraph", params=upper_parms, developerKey=GoogleAPIkey)
lower = get(method="getGraph", params=lower_parms, developerKey=GoogleAPIkey)

# test whether keyword is case-sensitive
assert upper["lines"][0]["points"] == lower["lines"][0]["points"], "Keyword is sensitive."

In [4]:
# Transform to df
kw_png = pd.json_normalize(upper["lines"],
                           meta=["term"],
                           record_path=["points"])
kw_png.head(5)

,value,date,term
0,23,2004-01-01,Papua New Guinea
1,100,2004-02-01,Papua New Guinea
2,73,2004-03-01,Papua New Guinea
3,56,2004-04-01,Papua New Guinea
4,72,2004-05-01,Papua New Guinea


## Downloading files

In [5]:
def multiple_request(country: str,
                     geos: list = ["AU", "NZ", "US", "JP", "HK", "NC"],
                     second_terms: list = ["flight", "travel", "hotel"]) -> pd.DataFrame:
    """
    Args:
        country -- an Pacific Islands Country name in string;
        geos -- a list object contains that ISO2 letter code;
        second_terms -- a list object contains secondary terms relevant to traveling;

    Returns: 
        pandas.DataFrame 
    """
    params = {
        "restrictions_startDate": "2004-01"
    }

    df = pd.DataFrame()
    df["date"] = pd.date_range(start="2004-01-01", end="2022-10-01", freq="MS")

    for geo in geos:
        params.update({"restrictions_geo": geo})

        for term in second_terms:
            search_term = str(country) + " " + str(term)
            params.update({"terms": search_term})

            geo_raw = get(method="getGraph", params=params,
                          developerKey=GoogleAPIkey)

            geo_df = pd.json_normalize(geo_raw["lines"],
                                       meta=["term"],
                                       record_path=["points"])

            geo_df["date"] = pd.to_datetime(geo_df["date"])

            geo_df = (geo_df.rename({"value": str(geo) + "-" + str(search_term)}, axis=1)
                      .drop("term", axis=1))
            df = pd.merge(df, geo_df, how="left", on="date")

    return df

In [15]:
fiji = multiple_request("fiji")
fiji.head(5)

,date,AU-fiji flight,AU-fiji travel,AU-fiji hotel,NZ-fiji flight,NZ-fiji travel,NZ-fiji hotel,US-fiji flight,US-fiji travel,US-fiji hotel,JP-fiji flight,JP-fiji travel,JP-fiji hotel,HK-fiji flight,HK-fiji travel,HK-fiji hotel,NC-fiji flight,NC-fiji travel,NC-fiji hotel
0,2004-01-01,87,100,0,82,100,0,0,65,51,0,0,0,0,0,0,0,0,0
1,2004-02-01,25,63,88,0,0,31,0,87,0,0,0,0,0,0,0,0,0,0
2,2004-03-01,35,0,53,0,73,0,0,72,100,0,0,0,0,0,0,0,0,0
3,2004-04-01,23,34,57,0,21,0,0,67,0,0,0,0,0,0,0,0,0,0
4,2004-05-01,0,52,71,38,21,32,37,43,27,0,0,0,0,98,0,0,0,0


In [44]:
from bokeh.palettes import Category20
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Legend


# output_file('fiji_month_by_country.html')
p = figure(height=600, width=1000,
           title="Google Trend Index By Month",
           x_axis_type="datetime",
           x_axis_label="Date",
           y_axis_label="Google Trend Index",
           tooltips=[("Date", "$Date"),
                     ("Search Index", "@$name")])

cols = fiji.columns[fiji.columns.str.startswith("AU")].tolist()
p.add_layout(Legend(), 'right')
source = ColumnDataSource(fiji)

for col, color in zip(cols, Category20[3]):
    # have to use different colnames for y-coords so tooltip can refer to @$name
    p.line('date', col, source=fiji, name=col, legend_label=col, color=color)

p.legend.label_text_font_size = '9pt'
p.legend.click_policy = "mute"
p.legend.location = "top_left"

show(p)